In [1]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot, show_result_ins
import mmcv
import numpy as np
from scipy import ndimage
import cv2 as cv
from math import *
import pandas as pd

import matplotlib.pyplot as plt
import json
from shapely.geometry import Point, Polygon

xray_type = 'T1_TG'
config_file = './cfg/T1_TG/solov2_r101_dcn_fpn_8gpu_3x_custom.py'
# download the checkpoint from model zoo and put it in `checkpoints/`
checkpoint_file = './weight/T1_TG/epoch.pth'

# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')

# test a single image
img = 'C01002101_CR_ST001_SE001_IM00001.png'
result = inference_detector(model, img)
show_result_ins(img, result, model.CLASSES, score_thr=0.25, out_file="demo_out.jpg")


#############################################################################################################


cur_result = result[0]
seg_label = cur_result[0]
seg_label = seg_label.cpu().numpy().astype(np.uint8)
cate_label = cur_result[1]
cate_label = cate_label.cpu().numpy()
score = cur_result[2].cpu().numpy()

vis_inds = score > 0.3
seg_label = seg_label[vis_inds]
num_mask = seg_label.shape[0]
cate_label = cate_label[vis_inds]
cate_score = score[vis_inds]



img = mmcv.imread(img)
img_show = img.copy()
h, w, _ = img.shape

blank_image = np.zeros((h,w,3), np.uint8)
blank_mask = np.zeros((h,w,3), np.uint8)

mask_density = []
direction_vector = []
for idx in range(num_mask):
    cur_mask = seg_label[idx, :, :]
    cur_mask = mmcv.imresize(cur_mask, (w, h))
    cur_mask = (cur_mask > 0.5).astype(np.int32)
    mask_density.append(cur_mask.sum())

orders = np.argsort(mask_density)
seg_label = seg_label[orders]
cate_label = cate_label[orders]
cate_score = cate_score[orders]


np.random.seed(42)
color_masks = [
    np.random.randint(0, 256, (1, 3), dtype=np.uint8)
    for _ in range(num_mask)
]

################################################################################################################
CLASSES = ('L_T1', 'L_T2', 'L_T3', 'R_T1', 'R_T2', 'R_T3', 'R_T4', 'L_T4') # T1
#CLASSES = ('L_AP1', 'L_AP2', 'L_AP3', 'L_AP4', 'L_AP5', 'L_AP6', 'L_AP7', 'L_AP8', 'R_AP1', 'R_AP2', 'R_AP3', 'R_AP4', 'R_AP5', 'R_AP6', 'R_AP7', 'R_AP8') # X1
#CLASSES = ('L_LAT2', 'L_LAT1', 'L_LAT3', 'L_LAT4', 'L_LAT5', 'L_LAT6') # X2L





#CLASSES = ('R_LAT2', 'R_LAT1', 'R_LAT3', 'R_LAT4', 'R_LAT5', 'R_LAT6')

#CLASSES = ('L_LOB1', 'L_LOB2')
#CLASSES = ('R_LOB1', 'R_LOB2')

#CLASSES = ('R_HAV1', 'R_HAV2', 'L_HAV1', 'L_HAV2')

#CLASSES = ('R_AWB_AP1', 'R_AWB_AP2', 'R_AWB_AP3', 'L_AWB_AP1', 'L_AWB_AP2', 'L_AWB_AP3')

#CLASSES = ('L_AWB_LAT1', 'L_AWB_LAT2')
#CLASSES = ('R_AWB_LAT1', 'R_AWB_LAT2')

#CLASSES = ('L_KWB_AP1', 'L_KWB_AP2', 'L_KWB_AP3', 'R_KWB_AP1', 'R_KWB_AP2', 'R_KWB_AP3')

#CLASSES = ('L_KWB_LAT1', 'L_KWB_LAT2')
#CLASSES = ('R_KWB_LAT1', 'R_KWB_LAT2')



keypoints_dict = {}

for idx in range(num_mask):
    class_name = idx
    idx = -(idx+1)
    cur_mask = seg_label[idx, :, :]
    cur_mask = mmcv.imresize(cur_mask, (w, h))
    cur_mask = (cur_mask > 0.5).astype(np.uint8)
    if cur_mask.sum() == 0:
        continue
    color_mask = color_masks[idx]
    blank_color_mask = color_masks[0]
    cur_mask_bool = cur_mask.astype(np.bool)
    img_show[cur_mask_bool] = img[cur_mask_bool] * 0.5 + color_mask * 0.5
    
    blank_image[cur_mask_bool] = img[cur_mask_bool] * 0.5 + blank_color_mask * 0.5 # draw mask image
    gray = cv.cvtColor(blank_image, cv.COLOR_BGR2GRAY)
    _, bw = cv.threshold(gray, 1, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
    blank_mask = bw
    blank_image = np.zeros((h,w,3), np.uint8) # initialize original image
    
    cv.namedWindow("class_name", cv.WINDOW_NORMAL)
    cv.resizeWindow("class_name", 1920, 1080)
    cv.imshow("class_name", blank_mask)
    cv.waitKey(0)
    #cv.imwrite(class_name+".png", blank_mask)
    cur_cate = cate_label[idx]
    if xray_type == 'X3_FLOL':
        if cur_cate == 2:
            cur_cate = 1
    if xray_type == 'X3_FLOR':
        if cur_cate == 1:
            cur_cate = 0
        if cur_cate == 3:
            cur_cate = 1
    if xray_type == 'X6_AWBLL' or xray_type == 'X6_AWBLR':
        if cur_cate == 2:
            cur_cate = 0
        if cur_cate == 3:
            cur_cate = 1
    if xray_type == 'X8_KWBLL' or xray_type == 'X8_KWBLR':
        if cur_cate == 2:
            cur_cate = 1
        if cur_cate == 3:
            cur_cate = 0
    if cur_cate < len(CLASSES):
        print(CLASSES[cur_cate])
        keypoints_dict[CLASSES[cur_cate]] = blank_mask



sort_keypoints = sorted(keypoints_dict.items())
print(sort_keypoints)
    




with open("C01002101_CR_ST001_SE001_IM00001.json", "r", encoding="utf8") as f:
    contents = f.read()
    json_data = json.loads(contents)

#w = json_data["imageWidth"]
#h = json_data["imageHeight"]

point_list = []
point_dict = {}
line_dict = {}
#for obj in json_data["shapes"]:
for obj in json_data["ArrayOfannotation_info"]:
    #label_n = obj['label']
    label_n = obj['xyvalue']['label_val']['preset_detail_name']
    if obj['objectname'] == "LabelPolyLine":
        for points in obj['xyvalue']['pos_list']:
            point_list.append([int(points['X']), int(points['Y'])])
        
        #print(point_list)
        mask1 = np.zeros((h,w),dtype = np.uint8)
        polygon1 = np.array(point_list)
        img = cv.fillPoly(mask1,[polygon1],(255,255,255))
        point_list.clear()
        point_dict[label_n] = img
    if obj['objectname'] == "LabelCenterLine":
        line_list = []
        val_x = obj['xyvalue']['start_c_pos']['X']
        val_y = obj['xyvalue']['start_c_pos']['Y']
        line_list.append([float(val_x),float(val_y)])
        val_x = obj['xyvalue']['end_c_pos']['X']
        val_y = obj['xyvalue']['end_c_pos']['Y']
        line_list.append([float(val_x),float(val_y)])
        line_dict[label_n] = line_list


target_sort_keypoints = sorted(point_dict.items())
print(target_sort_keypoints)
target_line_sort_keypoints = sorted(line_dict.items())
#print(target_line_sort_keypoints)





for base_i in range(len(CLASSES)):
    json_chk = 0
    for target_i in range(len(target_sort_keypoints)):
        detect_chk = 0
        if CLASSES[base_i] == target_sort_keypoints[target_i][0]:
            json_chk = 1
            for pre_i in range(len(sort_keypoints)):
                if sort_keypoints[pre_i][0] == target_sort_keypoints[target_i][0]:
                    detect_chk = 1
                    intersection = np.logical_and(target_sort_keypoints[target_i][1], sort_keypoints[pre_i][1])
                    union = np.logical_or(target_sort_keypoints[target_i][1], sort_keypoints[pre_i][1])
                    iou_score = np.sum(intersection) / np.sum(union)
                    print(sort_keypoints[pre_i][0],iou_score)
            if detect_chk == 0:
                print(target_sort_keypoints[target_i][0], 'detect missing')
    if json_chk == 0:
        print(CLASSES[base_i], 'json nothing')
        
        
        


    
    
    

def line_exist(xray_type, line_CLASSES, seg_CLASSES, sort_keypoints, target_line_sort_keypoints):
    detect_line_num = len(target_line_sort_keypoints)
    minus = 0
    for pre_i in range(len(sort_keypoints)):
        for eval_i in range(len(seg_CLASSES)):
            if sort_keypoints[pre_i][0] == seg_CLASSES[eval_i]:
                if detect_line_num == 0:
                    print(line_CLASSES[eval_i], 'Nothing')
                else:
                    if target_line_sort_keypoints[eval_i-minus][0] == line_CLASSES[eval_i]:
                        contours, hierarchy = cv.findContours(sort_keypoints[pre_i][1], cv.RETR_LIST, cv.CHAIN_APPROX_NONE)
                        line_bool_val = []
                        for k in range(len(contours)):
                            poly = Polygon(contours[k].squeeze(1))
                            s_p = Point(target_line_sort_keypoints[eval_i-minus][1][0])
                            e_p = Point(target_line_sort_keypoints[eval_i-minus][1][1])
                            if s_p.within(poly) and e_p.within(poly):
                                line_bool_val.append('True')
                            else:
                                line_bool_val.append('False')
                        if 'True' in line_bool_val:
                            print(line_CLASSES[eval_i], 'True')
                        else:
                            print(line_CLASSES[eval_i], 'False')
                        detect_line_num = detect_line_num - 1
                    
                    elif target_line_sort_keypoints[eval_i-minus][0] != line_CLASSES[eval_i]:
                        print(line_CLASSES[eval_i], 'Nothing')
                        minus = minus + 1





line_CLASSES = ()
seg_CLASSES = ()

if xray_type == 'X1_FSAP':
    line_CLASSES = ('L_V1', 'L_V2', 'L_V3', 'L_V4', 'L_V5', 'L_V6', 'L_V7', 'R_V1', 'R_V2', 'R_V3', 'R_V4', 'R_V5', 'R_V6', 'R_V7')
    seg_CLASSES = ('L_AP1', 'L_AP2', 'L_AP3', 'L_AP4', 'L_AP5', 'L_AP7', 'L_AP8', 'R_AP1', 'R_AP2', 'R_AP3', 'R_AP4', 'R_AP5', 'R_AP7', 'R_AP8')
        
if xray_type == 'X2_FSLR' or xray_type == 'X2_FSLL':
    if xray_type == 'X2_FSLL':
        line_CLASSES = ('L_V1', 'L_V2', 'L_V3', 'L_V4')
        seg_CLASSES = ('L_LAT1', 'L_LAT3', 'L_LAT4', 'L_LAT6')
    else:
        line_CLASSES = ('R_V1', 'R_V2', 'R_V3', 'R_V4')
        seg_CLASSES = ('R_LAT1', 'R_LAT3', 'R_LAT4', 'R_LAT6')

if xray_type == 'X4_HAV':
    line_CLASSES = ('L_V1', 'L_V2', 'R_V1', 'R_V2')
    seg_CLASSES = ('L_HAV1', 'L_HAV2', 'R_HAV1', 'R_HAV2')

if xray_type == 'X5_AWBAP':
    line_CLASSES = ('L_V1', 'R_V1')
    seg_CLASSES = ('L_AWB_AP1', 'R_AWB_AP1')

if xray_type == 'X6_AWBLL' or xray_type == 'X6_AWBLR':
    if xray_type == 'X6_AWBLL':
        line_CLASSES = ('L_V1',)
        seg_CLASSES = ('L_AWB_LAT1',)
    else:
        line_CLASSES = ('R_V1',)
        seg_CLASSES = ('R_AWB_LAT1',)

if xray_type == 'X7_KWBAP':
    line_CLASSES = ('L_V1', 'L_V2', 'R_V1', 'R_V2')
    seg_CLASSES = ('L_KWB_AP1', 'L_KWB_AP2', 'R_KWB_AP1', 'R_KWB_AP2')
    
if xray_type == 'X8_KWBLL' or xray_type == 'X8_KWBLR':
    if xray_type == 'X8_KWBLL':
        line_CLASSES = ('L_V1', 'L_V2')
        seg_CLASSES = ('L_KWB_LAT1', 'L_KWB_LAT2')
    else:
        line_CLASSES = ('R_V1', 'R_V2')
        seg_CLASSES = ('R_KWB_LAT1', 'R_KWB_LAT2')
    
if xray_type == 'T1_TG':
    line_CLASSES = ('L_V1', 'L_V2', 'L_V3', 'R_V1', 'R_V2', 'R_V3')
    seg_CLASSES = ('L_T1', 'L_T1', 'L_T2', 'R_T1', 'R_T1', 'R_T2')


if len(line_CLASSES) != 0 or len(seg_CLASSES) != 0:
    line_exist(xray_type, line_CLASSES, seg_CLASSES, sort_keypoints, target_line_sort_keypoints)
            
"""
if xray_type == 'T1_TG':
    line_CLASSES = ('L_V1', 'L_V2', 'L_V3', 'R_V1', 'R_V2', 'R_V3')
    add = 0
    minus = 0
    for i in range(6):
        if i == 1:
            add = -1
        if i == 3:
            add = 1
        if i == 4:
            add = 0
        contours, hierarchy = cv.findContours(sort_keypoints[i+add][1], cv.RETR_LIST, cv.CHAIN_APPROX_NONE)
        print(sort_keypoints[i+add][0])
        line_bool_val = []
        for k in range(len(contours)):
            poly = Polygon(contours[k].squeeze(1))
            s_p = Point(target_line_sort_keypoints[i-minus][1][0])
            e_p = Point(target_line_sort_keypoints[i-minus][1][1])
            if s_p.within(poly) and e_p.within(poly):
                line_bool_val.append('True')
            else:
                line_bool_val.append('False')
        print(line_CLASSES[i])
        if 'True' in line_bool_val:
            print('True')
        else:
            print('False')
        if target_line_sort_keypoints[i-minus][0] != line_CLASSES[i]:
            minus = minus + 1
"""         
            

c:\users\ycs\anaconda3\envs\solov2\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
c:\users\ycs\anaconda3\envs\solov2\lib\site-packages\torch\nn\functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
c:\users\ycs\anaconda3\envs\solov2\lib\site-packages\torch\nn\functional.py:3658: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/lib

[]
[('L_T1', array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)), ('L_T2', array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)), ('L_T3', array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)), ('L_T4', array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)), ('R_T1', array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       

"\nif xray_type == 'T1_TG':\n    line_CLASSES = ('L_V1', 'L_V2', 'L_V3', 'R_V1', 'R_V2', 'R_V3')\n    add = 0\n    minus = 0\n    for i in range(6):\n        if i == 1:\n            add = -1\n        if i == 3:\n            add = 1\n        if i == 4:\n            add = 0\n        contours, hierarchy = cv.findContours(sort_keypoints[i+add][1], cv.RETR_LIST, cv.CHAIN_APPROX_NONE)\n        print(sort_keypoints[i+add][0])\n        line_bool_val = []\n        for k in range(len(contours)):\n            poly = Polygon(contours[k].squeeze(1))\n            s_p = Point(target_line_sort_keypoints[i-minus][1][0])\n            e_p = Point(target_line_sort_keypoints[i-minus][1][1])\n            if s_p.within(poly) and e_p.within(poly):\n                line_bool_val.append('True')\n            else:\n                line_bool_val.append('False')\n        print(line_CLASSES[i])\n        if 'True' in line_bool_val:\n            print('True')\n        else:\n            print('False')\n        if tar

In [1]:
def drawAxis(img, p_, q_, color, scale):
  p = list(p_)
  q = list(q_)

  ## [visualization1]
  angle = atan2(p[1] - q[1], p[0] - q[0]) # angle in radians
  hypotenuse = sqrt((p[1] - q[1]) * (p[1] - q[1]) + (p[0] - q[0]) * (p[0] - q[0]))

  # Here we lengthen the arrow by a factor of scale
  q[0] = p[0] - scale * hypotenuse * cos(angle)
  q[1] = p[1] - scale * hypotenuse * sin(angle)
  cv.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv.LINE_AA)

  # create the arrow hooks
  p[0] = q[0] + 9 * cos(angle + pi / 4)
  p[1] = q[1] + 9 * sin(angle + pi / 4)
  cv.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv.LINE_AA)

  p[0] = q[0] + 9 * cos(angle - pi / 4)
  p[1] = q[1] + 9 * sin(angle - pi / 4)
  cv.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv.LINE_AA)
  ## [visualization1]

def getOrientation(pts, img):
  ## [pca]
  # Construct a buffer used by the pca analysis
  sz = len(pts)
  data_pts = np.empty((sz, 2), dtype=np.float64)
  for i in range(data_pts.shape[0]):
    data_pts[i,0] = pts[i,0,0]
    data_pts[i,1] = pts[i,0,1]

  # Perform PCA analysis
  mean = np.empty((0))
  mean, eigenvectors, eigenvalues = cv.PCACompute2(data_pts, mean)

  # Store the center of the object
  cntr = (int(mean[0,0]), int(mean[0,1]))
  ## [pca]

  ## [visualization]
  # Draw the principal components
  cv.circle(img, cntr, 3, (255, 0, 255), 2)
  p1 = (cntr[0] + 0.02 * eigenvectors[0,0] * eigenvalues[0,0], cntr[1] + 0.02 * eigenvectors[0,1] * eigenvalues[0,0])
  p2 = (cntr[0] - 0.02 * eigenvectors[1,0] * eigenvalues[1,0], cntr[1] - 0.02 * eigenvectors[1,1] * eigenvalues[1,0])
  drawAxis(img, cntr, p1, (255, 255, 0), 1)
  drawAxis(img, cntr, p2, (0, 0, 255), 5)

  angle = atan2(eigenvectors[0,1], eigenvectors[0,0]) # orientation in radians
  ## [visualization]

  # Label with the rotation angle
  #label = "  Rotation Angle: " + str(-int(np.rad2deg(angle)) - 90) + " degrees"
  #textbox = cv.rectangle(img, (cntr[0], cntr[1]-25), (cntr[0] + 250, cntr[1] + 10), (255,255,255), -1)
  #cv.putText(img, label, (cntr[0], cntr[1]), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv.LINE_AA)

  return angle

In [ ]:
cur_result = result[0]
seg_label = cur_result[0]
seg_label = seg_label.cpu().numpy().astype(np.uint8)
cate_label = cur_result[1]
cate_label = cate_label.cpu().numpy()
score = cur_result[2].cpu().numpy()

vis_inds = score > 0.3
seg_label = seg_label[vis_inds]
num_mask = seg_label.shape[0]
cate_label = cate_label[vis_inds]
cate_score = score[vis_inds]

#############################################################################################################

img = mmcv.imread(img)
img_show = img.copy()
h, w, _ = img.shape

blank_image = np.zeros((h,w,3), np.uint8)
blank_mask = np.zeros((h,w,3), np.uint8)

mask_density = []
direction_vector = []
for idx in range(num_mask):
    cur_mask = seg_label[idx, :, :]
    cur_mask = mmcv.imresize(cur_mask, (w, h))
    cur_mask = (cur_mask > 0.5).astype(np.int32)
    mask_density.append(cur_mask.sum())

orders = np.argsort(mask_density)
seg_label = seg_label[orders]
cate_label = cate_label[orders]
cate_score = cate_score[orders]


np.random.seed(42)
color_masks = [
    np.random.randint(0, 256, (1, 3), dtype=np.uint8)
    for _ in range(num_mask)
]
################################################################################################################

for idx in range(num_mask):
    idx = -(idx+1)
    cur_mask = seg_label[idx, :, :]
    cur_mask = mmcv.imresize(cur_mask, (w, h))
    cur_mask = (cur_mask > 0.5).astype(np.uint8)
    if cur_mask.sum() == 0:
        continue
    color_mask = color_masks[idx]
    blank_color_mask = color_masks[0]
    cur_mask_bool = cur_mask.astype(np.bool)
    img_show[cur_mask_bool] = img[cur_mask_bool] * 0.5 + color_mask * 0.5
    
    blank_image[cur_mask_bool] = img[cur_mask_bool] * 0.5 + blank_color_mask * 0.5 # draw mask image
    gray = cv.cvtColor(blank_image, cv.COLOR_BGR2GRAY)
    _, bw = cv.threshold(gray, 1, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
    blank_mask = bw
    contours, _ = cv.findContours(blank_mask, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)
    
    for i, c in enumerate(contours):
        area = cv.contourArea(c)
        #if area < 3700 or 60000 < area:
            #continue
        
        getOrientation(c, img_show)
    
    blank_image = np.zeros((h,w,3), np.uint8) # initialize original image
    #cv.imshow("", blank_mask)
    #cv.waitKey(0)
        
    cur_cate = cate_label[idx]
    cur_score = cate_score[idx]    
    center_y, center_x = ndimage.measurements.center_of_mass(cur_mask)
    top_y, top_x = ndimage.measurements.maximum_position(cur_mask) # top of segmentation
    #center_x = int(center_x)
    #center_y = int(center_y)
    #cv2.line(img_show, (300,300) , (center_x,center_y), (0,255,0), 1)
    
    direction_vector.append([top_x-center_x,top_y-center_y])
    #print(direction_vector)
    vis_pos = (max(int(center_x) - 10, 0), int(center_y))
    
    
    
    
contours, _ = cv.findContours(blank_mask, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)

for i, c in enumerate(contours):
    area = cv.contourArea(c)
    
    #if area < 3700 or 60000 < area:
        #continue
    
    getOrientation(c, img_show)
    #cv.imshow('test', img_show)
    #cv.waitKey(0)


u_1 = direction_vector[0]
u_2 = direction_vector[2]


cos_val = ((u_1[0]*u_2[0])+(u_1[1]*u_2[1]))/((sqrt(pow(u_1[0],2)+pow(u_1[1],2)))*(sqrt(pow(u_2[0],2)+pow(u_2[1],2))))
angle_val = degrees(acos(cos_val))
#print("angle: ",angle_val)

cv.imshow("", img_show)
cv.waitKey(0)
cv.imwrite("out.png", img_show)


c:\users\ycs\anaconda3\envs\open-mmlab\lib\site-packages\ipykernel_launcher.py:53: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
